In [2]:
import yaml
import os
import pprint
from copy import deepcopy
import pandas as pd
from pathlib import Path
from influence_moo.config import load_config, merge_dicts, expand_keys

In [4]:
base_config_dir = '~/influence-multi-objective/example_sweep/base_config.yaml'
sweep_config_dir = '~/influence-multi-objective/example_sweep/sweep.yaml'
top_dir = "~/influence-multi-objective/results/auto-generated/"

# Load in the base config
base_config = load_config(base_config_dir)

# Load in sweep config
sweep_config = load_config(sweep_config_dir)

In [10]:
# Expand the keys in the sweep config
expand_keys(sweep_config)

big_config = {}
for p in sweep_config['parameter_sets']:
    big_config[p] = {}
    for r in sweep_config['rewards']:
        # Make a copy of the base config
        new_config = deepcopy(base_config)
        # Merge in the specific parameter set for this config
        merge_dicts(new_config, sweep_config['parameter_sets'][p])
        # Merge in the reward parameters for this config
        merge_dicts(new_config, sweep_config['rewards'][r])
        big_config[p][r] = new_config

# Now turn this into a list of directories with the corresponding config we are going to save there
directory_dict = {}
for p in big_config:
    for r in big_config[p]:
        dir = Path(p) / r
        directory_dict[dir]  = big_config[p][r]

In [14]:
for dir in directory_dict:
    write_dir = Path(os.path.expanduser(top_dir)) / dir
    if not os.path.exists(write_dir):
        os.makedirs(write_dir, exist_ok = True)
    with open(write_dir/'config.yaml', 'w') as file:
        yaml.dump(directory_dict[dir], file, default_flow_style=False)  # `default_flow_style=False` makes the output more readable